<a href="https://colab.research.google.com/github/trevinofernando/Algorithms-for-Machine-Learning/blob/master/CreditCardFraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fraud Classifiers for Credit Card Transactions


In [3]:
import os
import json
import numpy as np
import pandas as pd
from enum import Enum 
import matplotlib.pyplot as plt
from google.colab import files,drive
from sklearn.model_selection import train_test_split


Globals and Enums

In [4]:
#Enums:
class AXIS(Enum):
    row = 0
    col = 1

Mount google drive to load data

In [5]:
#Mount to my google drive
drive.mount('/content/drive',force_remount=True)

os.chdir('/content/drive/My Drive/Colab Notebooks/Datasets/CreditCardFrauds')
!pwd

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Datasets/CreditCardFrauds


In [6]:
!ls

transactions.txt  transactions.zip


##Read dataset in json format into dataframe

In [7]:
# converts list of dictionary of intances into dictionary of features 
# and returns column oriented dataframe 
def load_data(data):
    parse_data=data[0].keys()
    trans_data={}
    for i in parse_data:
        trans_data[i]=list()
    for row in data:
        for keys,values in row.items():
            if values=='':
                trans_data[keys].append(np.nan) #missing datapoints are assigned with 'NaN'
            else:
                trans_data[keys].append(values)
    return pd.DataFrame.from_dict(trans_data)

In [8]:
#Opening the file
contents = open("transactions.txt", "r").read()  
list_dict = [json.loads(str(item)) for item in contents.strip().split('\n')]
original_transaction_df = load_data(list_dict)

#Transaction data Data Frame
original_transaction_df

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,isFraud,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch
0,733493772,733493772,5000.0,5000.00,2016-01-08T19:04:50,111.33,Lyft,US,US,05,01,rideshare,04/2020,2014-08-03,2014-08-03,492,492,9184,PURCHASE,True,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False
1,733493772,733493772,5000.0,4888.67,2016-01-09T22:32:39,24.75,Uber,US,US,09,01,rideshare,06/2023,2014-08-03,2014-08-03,492,492,9184,PURCHASE,False,NaN,111.33,NaN,NaN,NaN,False,NaN,NaN,False
2,733493772,733493772,5000.0,4863.92,2016-01-11T13:36:55,187.40,Lyft,US,US,05,01,rideshare,12/2027,2014-08-03,2014-08-03,492,492,9184,PURCHASE,False,NaN,136.08,NaN,NaN,NaN,False,NaN,NaN,False
3,733493772,733493772,5000.0,4676.52,2016-01-11T22:47:46,227.34,Lyft,US,US,02,01,rideshare,09/2029,2014-08-03,2014-08-03,492,492,9184,PURCHASE,True,NaN,323.48,NaN,NaN,NaN,False,NaN,NaN,False
4,733493772,733493772,5000.0,4449.18,2016-01-16T01:41:11,0.00,Lyft,US,US,02,01,rideshare,10/2024,2014-08-03,2014-08-03,492,492,9184,ADDRESS_VERIFICATION,False,NaN,550.82,NaN,NaN,NaN,False,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641909,186770399,186770399,7500.0,2574.02,2016-12-04T12:29:21,5.37,Apple iTunes,US,US,05,08,mobileapps,01/2030,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,NaN,4925.98,NaN,NaN,NaN,False,NaN,NaN,False
641910,186770399,186770399,7500.0,2568.65,2016-12-09T04:20:35,223.70,Blue Mountain eCards,US,US,09,01,online_gifts,05/2026,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,NaN,4931.35,NaN,NaN,NaN,False,NaN,NaN,False
641911,186770399,186770399,7500.0,2344.95,2016-12-16T07:58:23,138.42,Fresh Flowers,US,US,02,01,online_gifts,10/2019,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,NaN,5155.05,NaN,NaN,NaN,False,NaN,NaN,False
641912,186770399,186770399,7500.0,2206.53,2016-12-19T02:30:35,16.31,abc.com,US,US,09,08,online_subscriptions,11/2029,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,NaN,5293.47,NaN,NaN,NaN,False,NaN,NaN,False


##Exploring Dataset part 1

In [9]:
summary_df = original_transaction_df.describe(include = 'all')

summary_df.loc['dtype'] = original_transaction_df.dtypes
summary_df.loc['mode'] = original_transaction_df.mode().mode().iloc[0]
summary_df.loc['median'] = original_transaction_df.median()
summary_df.loc['NaN_%'] = original_transaction_df.isnull().mean()

summary_df

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,isFraud,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch
count,641914,641914,641914,641914,641914,641914,641914,638001,641290,638569,641627,641914,641914,641914,641914,641914,641914,641914,641325,641914,0,641914,0,0,0,641914,0,0,641914
unique,5000,5000,NaN,NaN,635472,NaN,2493,4,4,5,3,19,165,1826,2186,899,980,5134,3,2,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,2
top,318001076,318001076,NaN,NaN,2016-01-11T00:34:57,NaN,Lyft,US,US,05,01,online_retail,05/2026,2015-12-11,2016-07-20,633,633,1789,PURCHASE,False,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False
freq,10034,10034,NaN,NaN,3,NaN,25311,632303,635577,255615,514144,161469,4209,10137,3948,11354,11254,10034,608685,630612,NaN,NaN,NaN,NaN,NaN,340453,NaN,NaN,640945
mean,NaN,NaN,10697.2,6652.83,NaN,135.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4044.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,11460.4,9227.13,NaN,147.053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5945.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,250,-1244.93,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,5000,1114.97,NaN,32.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,502.442,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,7500,3578.16,NaN,85.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2151.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,15000,8169.18,NaN,189.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5005.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As seen above, the row 'NaN_%' shows that 6 columns are completly filled with NaN values (echoBuffer, merchantCity, etc.). And using the 'mode' to replace NaN values would make the most sense as long as it is used per account.

##Data Cleaning

###Handling columns missing a significant amount of data (80% or more)

In [10]:
transaction_df = original_transaction_df.copy()
for col in transaction_df:
  if summary_df.loc['NaN_%',col] >= 0.80:
    print("Dropping column '{}' for having {}% of NaN entries".format( col , summary_df.loc['NaN_%',col] * 100.0))
    transaction_df.drop(col, axis=AXIS.col.value, inplace=True)

print("Old dimensions: {}".format(original_transaction_df.shape))
print("New dimensions: {}".format(transaction_df.shape))
transaction_df


Dropping column 'echoBuffer' for having 100.0% of NaN entries
Dropping column 'merchantCity' for having 100.0% of NaN entries
Dropping column 'merchantState' for having 100.0% of NaN entries
Dropping column 'merchantZip' for having 100.0% of NaN entries
Dropping column 'posOnPremises' for having 100.0% of NaN entries
Dropping column 'recurringAuthInd' for having 100.0% of NaN entries
Old dimensions: (641914, 29)
New dimensions: (641914, 23)


,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,isFraud,currentBalance,cardPresent,expirationDateKeyInMatch
0,733493772,733493772,5000.0,5000.00,2016-01-08T19:04:50,111.33,Lyft,US,US,05,01,rideshare,04/2020,2014-08-03,2014-08-03,492,492,9184,PURCHASE,True,0.00,False,False
1,733493772,733493772,5000.0,4888.67,2016-01-09T22:32:39,24.75,Uber,US,US,09,01,rideshare,06/2023,2014-08-03,2014-08-03,492,492,9184,PURCHASE,False,111.33,False,False
2,733493772,733493772,5000.0,4863.92,2016-01-11T13:36:55,187.40,Lyft,US,US,05,01,rideshare,12/2027,2014-08-03,2014-08-03,492,492,9184,PURCHASE,False,136.08,False,False
3,733493772,733493772,5000.0,4676.52,2016-01-11T22:47:46,227.34,Lyft,US,US,02,01,rideshare,09/2029,2014-08-03,2014-08-03,492,492,9184,PURCHASE,True,323.48,False,False
4,733493772,733493772,5000.0,4449.18,2016-01-16T01:41:11,0.00,Lyft,US,US,02,01,rideshare,10/2024,2014-08-03,2014-08-03,492,492,9184,ADDRESS_VERIFICATION,False,550.82,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641909,186770399,186770399,7500.0,2574.02,2016-12-04T12:29:21,5.37,Apple iTunes,US,US,05,08,mobileapps,01/2030,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,4925.98,False,False
641910,186770399,186770399,7500.0,2568.65,2016-12-09T04:20:35,223.70,Blue Mountain eCards,US,US,09,01,online_gifts,05/2026,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,4931.35,False,False
641911,186770399,186770399,7500.0,2344.95,2016-12-16T07:58:23,138.42,Fresh Flowers,US,US,02,01,online_gifts,10/2019,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,5155.05,False,False
641912,186770399,186770399,7500.0,2206.53,2016-12-19T02:30:35,16.31,abc.com,US,US,09,08,online_subscriptions,11/2029,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,5293.47,False,False


###Remove duplicate columns

In [11]:
#Code snipets taken from: https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
#https://stackoverflow.com/questions/10710328/comparing-numpy-arrays-containing-nan

def nan_equal(a,b):
  try:
    np.testing.assert_equal(a,b)
  except AssertionError:
    return False
  return True

def duplicate_columns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []

    for t, v in groups.items():

        cols = frame[v].columns
        vs = frame[v]
        numCols = len(cols)

        for i in range(numCols):
            ia = vs.iloc[:,i].values
            for j in range(i+1, numCols):
                ja = vs.iloc[:,j].values
                if nan_equal(ia, ja):
                    print("Marking column '{}' as duplicate of column '{}'". format(cols[i], cols[j]))
                    dups.append(cols[i])
                    break

    return dups

In [12]:
print("Old dimensions: {}".format(transaction_df.shape))
dupl_cols = duplicate_columns(transaction_df)
print("Columns to be removed for being duplicates: {} ".format(dupl_cols))
transaction_df = transaction_df.drop(dupl_cols, axis=AXIS.col.value)
print("New dimensions: {}".format(transaction_df.shape))
transaction_df

Old dimensions: (641914, 23)
Marking column 'accountNumber' as duplicate of column 'customerId'
Columns to be removed for being duplicates: ['accountNumber'] 
New dimensions: (641914, 22)


,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,isFraud,currentBalance,cardPresent,expirationDateKeyInMatch
0,733493772,5000.0,5000.00,2016-01-08T19:04:50,111.33,Lyft,US,US,05,01,rideshare,04/2020,2014-08-03,2014-08-03,492,492,9184,PURCHASE,True,0.00,False,False
1,733493772,5000.0,4888.67,2016-01-09T22:32:39,24.75,Uber,US,US,09,01,rideshare,06/2023,2014-08-03,2014-08-03,492,492,9184,PURCHASE,False,111.33,False,False
2,733493772,5000.0,4863.92,2016-01-11T13:36:55,187.40,Lyft,US,US,05,01,rideshare,12/2027,2014-08-03,2014-08-03,492,492,9184,PURCHASE,False,136.08,False,False
3,733493772,5000.0,4676.52,2016-01-11T22:47:46,227.34,Lyft,US,US,02,01,rideshare,09/2029,2014-08-03,2014-08-03,492,492,9184,PURCHASE,True,323.48,False,False
4,733493772,5000.0,4449.18,2016-01-16T01:41:11,0.00,Lyft,US,US,02,01,rideshare,10/2024,2014-08-03,2014-08-03,492,492,9184,ADDRESS_VERIFICATION,False,550.82,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641909,186770399,7500.0,2574.02,2016-12-04T12:29:21,5.37,Apple iTunes,US,US,05,08,mobileapps,01/2030,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,4925.98,False,False
641910,186770399,7500.0,2568.65,2016-12-09T04:20:35,223.70,Blue Mountain eCards,US,US,09,01,online_gifts,05/2026,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,4931.35,False,False
641911,186770399,7500.0,2344.95,2016-12-16T07:58:23,138.42,Fresh Flowers,US,US,02,01,online_gifts,10/2019,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,5155.05,False,False
641912,186770399,7500.0,2206.53,2016-12-19T02:30:35,16.31,abc.com,US,US,09,08,online_subscriptions,11/2029,2015-11-04,2016-06-03,127,127,5432,PURCHASE,False,5293.47,False,False


###Handling entries with a missing label

In [13]:
#Definetly don't want to guess labels. So drop any entry with missing labels (column=isFraud)
numRows = transaction_df.shape[AXIS.row.value]
transaction_df.dropna(subset = ['isFraud'])
print("Number of rows with missing labels: {}".format(numRows - transaction_df.shape[AXIS.row.value]))
if(numRows - transaction_df.shape[0] == 0):
  print("* This dataset had no missing labels. No entries were removed.")
numRows = transaction_df.shape[AXIS.row.value]

Number of rows with missing labels: 0
* This dataset had no missing labels. No entries were removed.


###Replacing NaN's with mode of corresponding Customer ID

In [17]:
#Groupby customerId
customer_Ids = transaction_df['customerId'].unique()
print("This dataset contains {} different customer IDs.".format(customer_Ids.size))

dictOfTransByCustomerId = {}
for i in range(customer_Ids.size):
  dictOfTransByCustomerId[i] = transaction_df.loc[ transaction_df['customerId'] == customer_Ids[i] ]
len(dictOfTransByCustomerId.keys())

This dataset contains 5000 different customer IDs.


5000

In [15]:
#Replace NaN values with mode TODO: refactor code to do it by customerId
for col in transaction_df:
  if summary_df.loc['NaN_%',col] > 0.0:
    print("Replacing NaN entries on column'{}' with mode: {}".format(col, summary_df.loc['mode',col]))
    transaction_df[col].fillna(summary_df.loc['mode',col], inplace=True)

Replacing NaN entries on column'acqCountry' with mode: US
Replacing NaN entries on column'merchantCountryCode' with mode: US
Replacing NaN entries on column'posEntryMode' with mode: 05
Replacing NaN entries on column'posConditionCode' with mode: 01
Replacing NaN entries on column'transactionType' with mode: PURCHASE
